In [104]:
import pandas as pd
import re
import numpy as np

In [105]:
cincinnati = pd.read_csv('../data_sources/cincinnati/CINCINNATI.csv', encoding="utf-8", low_memory=False, usecols=['DATE_FROM', 'OFFENSE','SUSPECT_AGE'])

In [106]:
cincinnati.head(10)

,DATE_FROM,OFFENSE,SUSPECT_AGE
0,10/19/2023 06:00:00 PM,CRIMINAL DAMAGING/ENDANGERING,UNKNOWN
1,10/19/2023 06:00:00 PM,CRIMINAL DAMAGING/ENDANGERING,UNKNOWN
2,10/19/2023 05:58:00 PM,THEFT,UNKNOWN
3,10/19/2023 05:58:00 PM,THEFT,UNKNOWN
4,10/20/2023 08:00:00 AM,DOMESTIC VIOLENCE,UNKNOWN
5,10/19/2023 05:00:00 PM,THEFT,UNKNOWN
6,10/19/2023 05:00:00 PM,THEFT,UNKNOWN
7,10/19/2023 10:00:00 PM,CRIMINAL DAMAGING/ENDANGERING,UNKNOWN
8,10/20/2023 07:30:00 AM,ASSAULT,18-25
9,10/20/2023 06:05:00 AM,BREAKING AND ENTERING,UNKNOWN


In [107]:
cincinnati.describe(include='all')

,DATE_FROM,OFFENSE,SUSPECT_AGE
count,516804,516443,516828
unique,282645,202,9
top,06/04/2023 05:19:00 PM,THEFT,UNKNOWN
freq,136,174954,334143


In [108]:
cincinnati["OFFENSE"].unique()

array(['CRIMINAL DAMAGING/ENDANGERING', 'THEFT', 'DOMESTIC VIOLENCE',
       'ASSAULT', 'BREAKING AND ENTERING',
       'IMPROPERLY DISCHARGING FIREARM AT/INTO HABITATION/SCHOOL',
       'BURGLARY', 'TELEPHONE HARASSMENT', 'AGGRAVATED MENACING', 'RAPE',
       'ROBBERY', 'MURDER', 'CRIMINAL MISCHIEF', 'AGGRAVATED ROBBERY',
       'TAKING THE IDENTITY OF ANOTHER', 'PASSING BAD CHECKS',
       'UNAUTHORIZED USE OF MOTOR VEHICLE', 'FELONIOUS ASSAULT',
       'MAKING FALSE ALARMS', 'MENACING', nan, 'MENACING BY STALKING',
       'FAIL COMPLY ORDER/SIGNAL OF PO-ELUDE/FLEE',
       'MISUSE OF CREDIT CARD', 'ENDANGERING CHILDREN',
       'TELECOMMUNICATIONS FRAUD', 'UNAUTHORIZED USE OF PROPERTY',
       'VANDALISM', 'VIOLATE PROTECTION ORDER/CONSENT AGREEMENT',
       'SEXUAL IMPOSITION', 'FORGERY', 'ABDUCTION', 'NEGLIGENT ASSAULT',
       'SEXUAL BATTERY', 'AGGRAVATED BURGLARY',
       'INTERFERENCE WITH CUSTODY', 'KIDNAPPING', 'VEHICULAR VANDALISM',
       'INDUCING PANIC', 'GROSS SEXUAL IM

In [109]:
cincinnati["SUSPECT_AGE"].unique()

array(['UNKNOWN', '18-25', '31-40', '41-50', 'UNDER 18', '26-30', '61-70',
       '51-60', 'OVER 70'], dtype=object)

In [110]:
crime_dict = {
    'THEFT': 'Theft and Property Offenses',
    'CRIMINAL DAMAGING/ENDANGERING': 'Criminal Misconduct',
    'ASSAULT': 'Assault and Violent Crimes',
    'TELEPHONE HARASSMENT': 'Harassment and Intimidation',
    'BURGLARY': 'Theft and Property Offenses',
    'INTERFERENCE WITH CUSTODY': 'Child-Related Offenses',
    'AGGRAVATED MENACING': 'Assault and Violent Crimes',
    'AGGRAVATED ROBBERY': 'Theft and Property Offenses',
    'BREAKING AND ENTERING': 'Theft and Property Offenses',
    'DOMESTIC VIOLENCE': 'Assault and Violent Crimes',
    'MENACING': 'Harassment and Intimidation',
    'FELONIOUS ASSAULT': 'Assault and Violent Crimes',
    'TAMPERING WITH COIN MACHINES': 'Fraud and Deception',
    'ROBBERY': 'Theft and Property Offenses',
    'PASSING BAD CHECKS': 'Fraud and Deception',
    'SEXUAL IMPOSITION': 'Sexual Offenses',
    'TAKING THE IDENTITY OF ANOTHER': 'Fraud and Deception',
    'CRIMINAL MISCHIEF': 'Criminal Misconduct',
    'UNAUTHORIZED USE OF MOTOR VEHICLE': 'Traffic and Vehicle-Related Offenses',
    'RAPE': 'Sexual Offenses',
    'FAIL COMPLY ORDER/SIGNAL OF PO-ELUDE/FLEE': 'Traffic and Vehicle-Related Offenses',
    'FORGERY': 'Fraud and Deception',
    'INDUCING PANIC': 'Criminal Misconduct',
    'MURDER': 'Assault and Violent Crimes',
    'VIOLATE PROTECTION ORDER/CONSENT AGREEMENT': 'Criminal Misconduct',
    'UNLAWFUL RESTRAINT': 'Assault and Violent Crimes',
    'AGGRAVATED BURGLARY': 'Theft and Property Offenses',
    'UNLAWFUL SEXUAL CONDUCT WITH A MINOR': 'Sexual Offenses',
    'MENACING BY STALKING': 'Harassment and Intimidation',
    'EXTORTION': 'Criminal Misconduct',
    'SEXUAL BATTERY': 'Sexual Offenses',
    'VANDALISM': 'Theft and Property Offenses',
    'GROSS SEXUAL IMPOSITION': 'Sexual Offenses',
    'KIDNAPPING': 'Assault and Violent Crimes',
    'IMPROPERLY DISCHARGING FIREARM AT/INTO HABITATION/SCHOOL': 'Firearm and Weapon-Related Crimes',
    'PUBLIC INDECENCY': 'Miscellaneous Offenses',
    'TELECOMMUNICATIONS FRAUD': 'Fraud and Deception',
    'ENDANGERING CHILDREN': 'Child-Related Offenses',
    'INTIMIDATION-PUBLIC SERV/PARTY OFF': 'Harassment and Intimidation',
    'AGGRAVATED ASSAULT': 'Assault and Violent Crimes',
    'UNAUTHORIZED USE OF PROPERTY': 'Theft and Property Offenses',
    'MISUSE OF CREDIT CARD': 'Fraud and Deception',
    'MAKING FALSE ALARMS': 'Miscellaneous Offenses',
    'VEHICULAR VANDALISM': 'Theft and Property Offenses',
    'PATIENT ABUSE; NEGLECT': 'Child-Related Offenses',
    'ABDUCTION': 'Child-Related Offenses',
    'RUNAWAY': 'Miscellaneous Offenses',
    'NEGLIGENT ASSAULT': 'Assault and Violent Crimes',
    'AGGRAVATED TRESPASS': 'Theft and Property Offenses',
    'INTIMIDATION': 'Harassment and Intimidation',
    'WIRE FRAUD': 'Fraud and Deception',
    'IMPORTUNING': 'Miscellaneous Offenses',
    'CRIMINAL CHILD ENTICEMENT': 'Child-Related Offenses',
    'AGGRAVATED VEHICULAR ASSAULT': 'Assault and Violent Crimes',
    'VOYEURISM': 'Sexual Offenses',
    'B&E-COMMIT FELONY-PREMISES OF ANOTHER': 'Theft and Property Offenses',
    'EMBEZZLEMENT': 'Fraud and Deception',
    'SAFECRACKING': 'Theft and Property Offenses',
    'DISRUPTING PUBLIC SERVICE': 'Miscellaneous Offenses',
    'AGG VEH HOM/VEH HOM/VEH MNSLGHTER': 'Assault and Violent Crimes',
    'NEGLIGENT HOMICIDE': 'Assault and Violent Crimes',
    'INVOLUNTARY MANSLAUGHTER': 'Assault and Violent Crimes',
    'ARSON': 'Firearm and Weapon-Related Crimes',
    'FAIL TO PROVIDE FOR FUNCTIONALLY IMPAIRED PERSON': 'Child-Related Offenses',
    'RECKLESS HOMICIDE': 'Assault and Violent Crimes',
    'ETHNIC INTIMIDATION': 'Harassment and Intimidation',
    'CRIMINAL SIMULATION': 'Miscellaneous Offenses',
    'IMPERSONATING PEACE OFFICER/PRIVATE POLICEMAN': 'Fraud and Deception',
    'AGGRAVATED MURDER': 'Assault and Violent Crimes',
    'COERCION': 'Criminal Misconduct',
    'BRIBERY': 'Criminal Misconduct',
    'HAZING': 'Miscellaneous Offenses',
    'PERSONATING AN OFFICER': 'Criminal Misconduct',
    'RETALIATION': 'Miscellaneous Offenses',
    'CRIMINAL DAMAGING/ENDANGERING - KNOWINGLY, ANY MEANS': 'Criminal Misconduct',
    'VOLUNTARY MANSLAUGHTER': 'Assault and Violent Crimes',
    'DEFRAUDING A LIVERY OR HOSTELRY': 'Fraud and Deception',
    'UNAUTHORIZED USE OF MOTOR VEHICLE-JOY RIDING': 'Theft and Property Offenses',
    'SEXUAL ASSAULT WITH AN OBJECT': 'Sexual Offenses',
    'THEFT- WITHOUT CONSENT     M1F4': 'Theft and Property Offenses',
    'DISSEMINATE MATTER HARMFUL TO JUVENILES': 'Child-Related Offenses',
    'CONTAMINATE SUBST FOR HUMAN CONSUMP/USE,SPREAD FALSE REPORT': 'Criminal Misconduct',
    'FAILURE TO COMPLY ORDER/SIGNAL OF PO': 'Miscellaneous Offenses',
    'OBSTRUCTING OFFICIAL BUSINESS': 'Miscellaneous Offenses',
    'CRIMINAL TRESPASS': 'Theft and Property Offenses',
    'AGGRAVATED MURDER - PREMEDITATED': 'Assault and Violent Crimes',
    'RECEIVING STOLEN PROPERTY': 'Theft and Property Offenses',
    'THEFT- BEYOND SCOPE OF CNSNT M1F4': 'Theft and Property Offenses',
    'RAPE - VICTIM < 13, NON-FORCIBLE': 'Sexual Offenses',
    'FORGERY - UTTER, POSSESS TO UTTER': 'Fraud and Deception',
    'RAPE - SUBSTANTIALLY IMPAIR JUDGMENT': 'Sexual Offenses',
    'DISORDERLY CONDUCT - INTERFERING WITH OTHERS': 'Miscellaneous Offenses',
    'AGGRAVATED ARSON': 'Firearm and Weapon-Related Crimes',
    'CONTRIB TO CHILD UNRULINESS/DELINQUENCY': 'Child-Related Offenses',
    'TELEPHONE HARASSMENT - ANONYMOUS': 'Harassment and Intimidation',
    'PUBLIC INDECENCY - EXPOSURE': 'Miscellaneous Offenses',
    'THEFT BY DECEPTION        M1F4': 'Fraud and Deception',
    'GROSS SEXUAL IMPOSITION - < 13 YRS, STATUTORY': 'Sexual Offenses',
    'DISORDERLY CONDUCT-FIGHTING/THREATENING': 'Assault and Violent Crimes',
    'DISORDERLY CONDUCT': 'Miscellaneous Offenses',
    'FEL ASSLT - SEXUAL CONDUCT W/O DISCLOSING HIV KNOWLEDGE': 'Sexual Offenses',
    'SEXUAL IMPOSITION - VICTIM 13, 14, 15': 'Sexual Offenses',
    'B&E -COMMIT THEFT/FELONY-UNOCC/FORCE/STEALTH': 'Theft and Property Offenses',
    'FELONIOUS ASSAULT - WEAPON OR ORDNANCE': 'Assault and Violent Crimes',
    'ENDANGERING CHILDREN - ABUSE': 'Child-Related Offenses',
    'ENDANGERING CHILDREN-RISK HARM': 'Child-Related Offenses',
    'ASSAULT - KNOWINGLY HARM VICTIM': 'Assault and Violent Crimes',
    'IMPROPERLY DISCHARGING FIREARM AT/INTO HABITATION/SCHOOL-OCCUPIED STRUCTURE': 'Firearm and Weapon-Related Crimes',
    'TAMPERING WITH RECORDS': 'Criminal Misconduct',
    'IMPERSONATING PO/PRIVATE POLICEMAN-FACILITATE CRIME': 'Criminal Misconduct',
    'THEFT - BY INTIMIDATION': 'Theft and Property Offenses',
    'AGGRAVATED BURGLARY - INFLICT HARM': 'Theft and Property Offenses',
    'TELEPHONE HARASS-FAIL TO DESIST': 'Harassment and Intimidation',
    'GROSS SEX IMP - VICT MENTAL/PHYSICAL COND': 'Sexual Offenses',
    'SEXUAL BATTERY - VICTIM COERCED': 'Sexual Offenses',
    'FELONIOUS ASSAULT - VICTIM SERIOUSLY HARMED': 'Assault and Violent Crimes',
    'SEX BAT - VIC MINOR, OFF TMP/OCC DISCIP CONTR': 'Sexual Offenses',
    'ABDUCTION - REMOVE FROM PLACE FOUND': 'Child-Related Offenses',
    'DOMESTIC VIOLENCE - KNOWINGLY HARM': 'Assault and Violent Crimes',
    'VANDALISM - GOVERNMENT ENTITY': 'Theft and Property Offenses',
    'AGG ROBBERY - ARMED, DEADLY WEAPON': 'Theft and Property Offenses',
    'BURGLARY- TRESPASS STRUCT TO COMMIT OFFENSE': 'Theft and Property Offenses',
    'GROSS SEXUAL IMPOSITION - FORCE': 'Sexual Offenses',
    'ABDUCTION - RESTRAIN LIBERTY': 'Child-Related Offenses',
    'THEFT BY THREAT           M1F4': 'Theft and Property Offenses',
    'ROBBERY - USE/THREATEN IMMED USE OF FORCE': 'Theft and Property Offenses',
    'SEXUAL BATTERY - PARENT OR GUARDIAN': 'Sexual Offenses',
    'THEFT IN OFFICE': 'Theft and Property Offenses',
    'SEXUAL IMPOSITION - VICTIM IMPAIRED': 'Sexual Offenses',
    'AGG BURGLARY-ARMED W/ DEADLY WEAPON, ORDNANCE': 'Theft and Property Offenses',
    'BURGLARY- TRESPASS OCC/LIKELY OCC STRUCT TO COMMIT OFFENSE': 'Theft and Property Offenses',
    'SEXUAL IMPOSITION - OFFENSIVE CONTACT': 'Sexual Offenses',
    'BURGLARY- TRESPASS LIKELY OCC STRUCT COMMIT OFFENSE': 'Theft and Property Offenses',
    'AGGRAVATED ASSAULT - WEAPON OR ORDNANCE': 'Assault and Violent Crimes',
    'VIOL PROT ORDER/CONSENT AGREE-2903-203/2903-204': 'Criminal Misconduct',
    'DECEPTION TO OBTAIN A DANGEROUS DRUG': 'Fraud and Deception',
    'FALSIFICATION': 'Criminal Misconduct',
    'RESISTING ARREST': 'Miscellaneous Offenses',
    'ENDANGERING CHILDREN-CORPORAL PUNISHMENT': 'Child-Related Offenses',
    'CRIMINAL MISCHIEF - MOVE/DEFACE/TAMPER PROP OF ANOTHER': 'Criminal Misconduct',
    'AGGRAVATED MURDER - FELONY': 'Assault and Violent Crimes',
    'VIOL PROT ORDER/CONSENT AGREE-2919-26/3113-31': 'Criminal Misconduct',
    'RAPE - FORCE, THREAT OF': 'Sexual Offenses',
    'ILLEGAL USE OF MINOR IN NUDITY ORIENTED MATERIAL': 'Child-Related Offenses',
    'DISORDERLY CONDUCT-PHYS OFFENSIVE COND': 'Miscellaneous Offenses',
    'POSSESSING CRIMINAL TOOLS': 'Criminal Misconduct',
    'KIDNAPPING - COMMISSION OF FELONY, FLIGHT': 'Child-Related Offenses',
    'DISCHARGE FIREARM ON/NEAR PROHIBITED PREMISES': 'Firearm and Weapon-Related Crimes',
    'PANDERING OBSCENITY INVOLVING MINOR': 'Sexual Offenses',
    'FORGERY - COPY, NOT GENUINE': 'Fraud and Deception',
    'DIS MATTER HARMFUL TO JUV - ALLOW TO REVIEW': 'Child-Related Offenses',
    'ROBBERY-INFLICT/ATTEMPT/THREAT SER PHYS HARM': 'Theft and Property Offenses',
    'TELEPHONE HARASS-ABUSE/THREATEN/ANNOY': 'Harassment and Intimidation',
    'VIOL PROT ORDER/CONSENT AGREE-OTHER STATE': 'Criminal Misconduct',
    'INTER W/ CUSTODY-<18, MENTALLY HANDICAPPED <21': 'Child-Related Offenses',
    'HAVING WEAPONS WHILE UNDER DISABILITY': 'Firearm and Weapon-Related Crimes',
    'ILLEGAL POSS FIREARM IN LIQUOR PERMIT PREMISES': 'Firearm and Weapon-Related Crimes',
    'PARENTAL EDUCATION NEGLECT': 'Child-Related Offenses',
    'KIDNAPPING - ENGAGE IN SEXUAL ACTIVITY': 'Child-Related Offenses',
    'KIDNAPPING - RESTRAIN LIBERTY': 'Child-Related Offenses',
    'DOMESTIC VIOLENCE - BELIEF IMMINENT HARM': 'Assault and Violent Crimes',
    'INTIMIDATION-VICTIM/WITNESS': 'Harassment and Intimidation',
    'DISORDERLY CONDUCT-OFFENSIVE GEST/NOISE': 'Miscellaneous Offenses',
    'ENDANGERING CHILDREN-TORTURE/CRUELLY ABUSE': 'Child-Related Offenses',
    'ROBBERY - DANGEROUS WEAPON, ORDNANCE': 'Theft and Property Offenses',
    'AGG ROBBERY - INFLICT/ATTEMPT SERIOUS HARM': 'Theft and Property Offenses',
    'UNAUTHORIZED USE OF PROPERTY-GENERAL': 'Theft and Property Offenses',
    'FORGERY - WITHOUT AUTHORITY': 'Fraud and Deception',
    'UNLAWFUL USE TELECOM DEVICE': 'Fraud and Deception',
    'DEFRAUDING LIVERY/HOSTELRY-FAILED TO PAY': 'Fraud and Deception',
    'HAVING WEAPONS UNDER DISABILITY-FUGITIVE FROM JUSTICE': 'Firearm and Weapon-Related Crimes',
    'SEXUAL BATTERY - MISTAKE FOR SPOUSE': 'Sexual Offenses',
    'PATIENT ABUSE; NEGLECT - GROSS NEGLECT': 'Child-Related Offenses',
    'MISUSE OF CREDIT CARD - OBTAIN BY DECEPTION': 'Fraud and Deception',
    'BURGLARY- TRESPASS OCC STRUCT TO COMMIT OFFENSE': 'Theft and Property Offenses',
    'DEFRAUDING CREDITORS': 'Fraud and Deception',
    'ASSAULT - RECKLESSLY HARM VICTIM': 'Assault and Violent Crimes',
    'VANDALISM - PROPERTY USED FOR BUSINESS > $500': 'Theft and Property Offenses',
    'CRIMINAL DAMAGING/ENDANGERING- RECKLESSLY,FIRE,FLOOD,EXPLODE': 'Criminal Misconduct',
    'AGG ROBBERY - ARMED, DEADLY ORDNANCE': 'Theft and Property Offenses',
    'PUBLIC INDECENCY - APPEAR TO BE SEX ACT': 'Miscellaneous Offenses',
    'INSURANCE FRAUD': 'Fraud and Deception',
    'UNATHORIZED USE OF CABLE DEVICE': 'Fraud and Deception',
    'INTIMID-VICTIM/WITNESS FORCE/THREAT': 'Harassment and Intimidation',
    'PUBLIC INDECENCY - ENGAGE IN SEX ACT': 'Miscellaneous Offenses',
    'KIDNAPPING - RANSOM, SHIELD, HOSTAGE': 'Child-Related Offenses',
    'VECHICULAR VANDALISM': 'Theft and Property Offenses',
    'COMPLICITY': 'Miscellaneous Offenses',
    "CRIMINAL MISCHIEF - SET FIRE OTHER'S LAND": 'Criminal Misconduct',
    'DISCH FIREARM ON/NEAR PROH PREM-SCHOOL/CHURCH/DWELLING OF ANOTHER/CHARIT INSTIT': 'Firearm and Weapon-Related Crimes',
    'SEXUAL BATTERY - SCHOOL PERSON OF AUTHORITY': 'Sexual Offenses',
    'ARSON - PROPERTY OF ANOTHER': 'Firearm and Weapon-Related Crimes',
    'POSS/SELL UNAUTHOR CABLE TELEV SERV-POSSESS': 'Fraud and Deception',
    'DEFR LIVERY/HOSTELRY-HIRED/RENTED W/O MEANS PAY': 'Fraud and Deception',
    'PATIENT ABUSE; NEGLECT - ABUSE': 'Child-Related Offenses',
    'DEFR LIVERY/HOSTELRY-ABSCONDED W/O PAYING': 'Fraud and Deception',
    'CRIMINAL MISCHIEF - TAMPER WITH MARKER': 'Criminal Misconduct',
    'RAPE - VICTIM MENTAL OR PHYSICAL DISABILITY': 'Sexual Offenses',
    'ENDANGERING CHILDREN-OP VEH UNDER INFL CHILD<18': 'Child-Related Offenses',
    'INVOL MANSLAUGHTER - RESULT OF MISDEMEANOR': 'Assault and Violent Crimes',
    'MAKING FALSE ALARMS-INITIATE REPORT/WARNING': 'Miscellaneous Offenses',
    'TRAFFICKING IN DRUGS-SELL/OFFER TO SELL': 'Criminal Misconduct',
    'ILLEGAL USE OF MINOR IN NUDITY ORIENTED MATERIAL': 'Child-Related Offenses',
}


In [111]:
cincinnati["Crime"] = cincinnati["OFFENSE"].map(crime_dict)

In [112]:
crime_categories = set(crime_dict.values())

# Вивести унікальні категорії
print("Унікальні категорії злочинів:")
for category in crime_categories:
    print(category)

# Порахувати кількість унікальних категорій
кількість_категорій = len(crime_categories)
print(f"\nЗагальна кількість унікальних категорій: {кількість_категорій}")


Унікальні категорії злочинів:
Miscellaneous Offenses
Sexual Offenses
Child-Related Offenses
Harassment and Intimidation
Assault and Violent Crimes
Traffic and Vehicle-Related Offenses
Fraud and Deception
Theft and Property Offenses
Criminal Misconduct
Firearm and Weapon-Related Crimes

Загальна кількість унікальних категорій: 10


In [113]:
cincinnati.rename(columns={'DATE_FROM' : 'Date', 'OFFENSE' : 'Specific crime'}, inplace=True)

In [114]:
cincinnati['City'] = 'Cincinnati'

In [115]:
cincinnati.head()

,Date,Specific crime,SUSPECT_AGE,Crime,City
0,10/19/2023 06:00:00 PM,CRIMINAL DAMAGING/ENDANGERING,UNKNOWN,Criminal Misconduct,Cincinnati
1,10/19/2023 06:00:00 PM,CRIMINAL DAMAGING/ENDANGERING,UNKNOWN,Criminal Misconduct,Cincinnati
2,10/19/2023 05:58:00 PM,THEFT,UNKNOWN,Theft and Property Offenses,Cincinnati
3,10/19/2023 05:58:00 PM,THEFT,UNKNOWN,Theft and Property Offenses,Cincinnati
4,10/20/2023 08:00:00 AM,DOMESTIC VIOLENCE,UNKNOWN,Assault and Violent Crimes,Cincinnati


In [116]:
cincinnati['Date'] = pd.to_datetime(cincinnati['Date'], format="%m/%d/%Y %H:%M:%S %p" , errors='coerce')
cincinnati['Date'] = cincinnati['Date'].dt.strftime('%d/%m/%Y')

In [117]:
cincinnati = cincinnati[["Date", "City", "Crime", "Specific crime"]]

In [118]:
cincinnati.head(10)

,Date,City,Crime,Specific crime
0,19/10/2023,Cincinnati,Criminal Misconduct,CRIMINAL DAMAGING/ENDANGERING
1,19/10/2023,Cincinnati,Criminal Misconduct,CRIMINAL DAMAGING/ENDANGERING
2,19/10/2023,Cincinnati,Theft and Property Offenses,THEFT
3,19/10/2023,Cincinnati,Theft and Property Offenses,THEFT
4,20/10/2023,Cincinnati,Assault and Violent Crimes,DOMESTIC VIOLENCE
5,19/10/2023,Cincinnati,Theft and Property Offenses,THEFT
6,19/10/2023,Cincinnati,Theft and Property Offenses,THEFT
7,19/10/2023,Cincinnati,Criminal Misconduct,CRIMINAL DAMAGING/ENDANGERING
8,20/10/2023,Cincinnati,Assault and Violent Crimes,ASSAULT
9,20/10/2023,Cincinnati,Theft and Property Offenses,BREAKING AND ENTERING


In [119]:
cincinnati.to_csv('../data_standartized/cincinnati.csv', index=False)